In [21]:
parallel = False
# specify the number of seeds
n_seeds = 3
# specify the datasets
datasets = ["cifar10", "cifar100"]
# specify the methods
methods = [
    # "fixmatch",
    # "freematch",
    "fixmatch-ours",
    # "freematch-ours",
    # "fixmatch-bam",
    # "freematch-bam",
    # "fixmatch-mr",
    # "freematch-mr",
]
# specify the number of iterations
n_iterations_map = {
    "cifar10": {
        "fixmatch": 37000,
        "freematch": 37000,
        "fixmatch-ours": 25000,
        "freematch-ours": 25000,
        "fixmatch-bam": 37000,
        "freematch-bam": 37000,
        "fixmatch-mr": 37000,
        "freematch-mr": 37000,
    },
    "cifar100": {
        "fixmatch": 70000,
        "freematch": 70000,
        "fixmatch-ours": 25000,
        "freematch-ours": 25000,
        "fixmatch-bam": 70000,
        "freematch-bam": 70000,
        "fixmatch-mr": 70000,
        "freematch-mr": 70000,
    },
    "svhn": {
        "fixmatch": 145000,
        "freematch": 145000,
        "fixmatch-ours": 25000,
        "freematch-ours": 25000,
        "fixmatch-bam": 145000,
        "freematch-bam": 145000,
        "fixmatch-mr": 145000,
        "freematch-mr": 145000,
    },
}
post_hoc_params = {
    "cifar10": {"post_hoc_eps": [0.001, 0.005], "post_hoc_max_epocs": 100},
    "cifar100": {
        "post_hoc_eps": [0.001, 0.005],
        "post_hoc_max_epocs": 500,
    },
    "svhn": {"post_hoc_eps": [0.001, 0.005], "post_hoc_max_epocs": 100},
}

In [22]:
n_th_map = {"cifar10": 1000, "cifar100": 3000, "stl10": 2400, "svhn": 3000}
n_cal_map = {"cifar10": 1000, "cifar100": 3000, "stl10": 2400, "svhn": 3000}
n_l_map = {"cifar10": 250, "cifar100": 2500, "stl10": 250, "svhn": 250}

In [23]:
scripts = []
lines = []
for dataset in datasets:

    n_th = n_th_map[dataset]
    n_cal = n_cal_map[dataset]
    n_l = n_l_map[dataset]

    for method in methods:

        n_iterations = n_iterations_map[dataset][method]

        for seed in range(n_seeds):
            for post_hoc_eps in post_hoc_params[dataset]["post_hoc_eps"]:

                pure_method = method.split("-")[0]

                should_use_cache = "True" if "ours" in method else "False"
                accumulate_pseudo_labels = "True" if "ours" in method else "False"
                use_post_hoc_calib = "True" if "ours" in method else "False"
                full_pl_flag = "True" if "ours" in method else "False"
                batch_pl_flag = "False" if "ours" in method else "True"
                take_d_cal_th_from = "eval" if "ours" in method else "train_lb"
                post_hoc_max_epocs = post_hoc_params[dataset]["post_hoc_max_epocs"]
                mr = "True" if "mr" in method else "False"
                bam = "True" if "bam" in method else "False"
                prefix = (
                    f"{method}_{dataset}_{seed}_{post_hoc_eps}"
                    if "ours" in method
                    else f"{method}_{dataset}_{seed}"
                )
                config = f"config/classic_cv/{pure_method}/{pure_method}_{dataset}_{n_l}_0.yaml"

                line = f"{config}, {prefix}, {seed}, {n_iterations}, {should_use_cache}, {accumulate_pseudo_labels}, {use_post_hoc_calib}, {full_pl_flag}, {batch_pl_flag}, {post_hoc_eps}, {post_hoc_max_epocs}, {mr}, {bam}, {take_d_cal_th_from}, {n_th}, {n_cal}"
                lines.append(line)

                script = f"OPT_TH=YES MODEL_SELECTION=Cov python3 train.py --c {config} --save_name determine --prefix {prefix} --gpu 0 --seed {seed} --epoch 1000 --num_train_iter {n_iterations} --should_use_cache {should_use_cache} --accumulate_pseudo_labels {accumulate_pseudo_labels} --use_post_hoc_calib {use_post_hoc_calib} --full_pl_flag {full_pl_flag} --batch_pl_flag {batch_pl_flag} --save_dir saved_models --post_hoc_eps {post_hoc_eps} --post_hoc_max_epochs {post_hoc_max_epocs} --mr {mr} --mr_config ./config/mr/default.yaml --bam {bam} --bam_config ./config/bam/default.yaml --take_d_cal_th_from {take_d_cal_th_from} --n_th {n_th} --n_cal {n_cal}"
                script += "&" if parallel else ""
                scripts.append(script)

In [24]:
len(scripts)

1

In [25]:
with open("run.sh", "w") as f:
    f.write("set -e\n")
    f.write("\n".join(scripts))

In [26]:
with open("args.txt", "w") as f:
    f.write("\n".join(lines))

In [27]:
"$1_$2_$3_$4_$5_$6_$7_$8_$9_${10}_${11}_${12}_${13}_${14}_${15}_${16}"

'$1_$2_$3_$4_$5_$6_$7_$8_$9_${10}_${11}_${12}_${13}_${14}_${15}_${16}'

In [28]:
"$(config) $(prefix) $(seed) $(n_iterations) $(should_use_cache) $(accumulate_pseudo_labels) $(use_post_hoc_calib) $(full_pl_flag) $(batch_pl_flag) $(post_hoc_eps) $(post_hoc_max_epocs) $(mr) $(bam) $(take_d_cal_th_from) $(n_th) $(n_cal)"

'$(config) $(prefix) $(seed) $(n_iterations) $(should_use_cache) $(accumulate_pseudo_labels) $(use_post_hoc_calib) $(full_pl_flag) $(batch_pl_flag) $(post_hoc_eps) $(post_hoc_max_epocs) $(mr) $(bam) $(take_d_cal_th_from) $(n_th) $(n_cal)'

In [29]:
"config, prefix, seed, n_iterations, should_use_cache, accumulate_pseudo_labels, use_post_hoc_calib, full_pl_flag, batch_pl_flag, post_hoc_eps, post_hoc_max_epocs, mr, bam, take_d_cal_th_from, n_th, n_cal"

'config, prefix, seed, n_iterations, should_use_cache, accumulate_pseudo_labels, use_post_hoc_calib, full_pl_flag, batch_pl_flag, post_hoc_eps, post_hoc_max_epocs, mr, bam, take_d_cal_th_from, n_th, n_cal'

In [30]:
"OPT_TH=YES MODEL_SELECTION=Cov python3 --c ${1} --save_name determine --prefix ${2} --gpu 0 --seed ${3} --epoch 1000 --num_train_iter ${4} --should_use_cache ${5} --accumulate_pseudo_labels ${6} --use_post_hoc_calib ${7} --full_pl_flag ${8} --batch_pl_flag ${9} --save_dir saved_models --post_hoc_eps ${10} --post_hoc_max_epochs ${11} --mr ${12} --mr_config ./config/mr/default.yaml --bam ${13} --bam_config ./config/bam/default.yaml --take_d_cal_th_from ${14} --n_th ${15} --n_cal ${16}"

'OPT_TH=YES MODEL_SELECTION=Cov python3 --c ${1} --save_name determine --prefix ${2} --gpu 0 --seed ${3} --epoch 1000 --num_train_iter ${4} --should_use_cache ${5} --accumulate_pseudo_labels ${6} --use_post_hoc_calib ${7} --full_pl_flag ${8} --batch_pl_flag ${9} --save_dir saved_models --post_hoc_eps ${10} --post_hoc_max_epochs ${11} --mr ${12} --mr_config ./config/mr/default.yaml --bam ${13} --bam_config ./config/bam/default.yaml --take_d_cal_th_from ${14} --n_th ${15} --n_cal ${16}'